# Mid-term project 20171255 Ha Sukyeon

read function

In [1]:
# The function to read sequences, make a dict, from a FASTA file
def read(tmp_filename):
    rv = dict()
    f = open(tmp_filename, 'r')
    for line in f:
        if line.startswith('>'):
            tmp_h = line.strip().lstrip('>')
            rv[tmp_h] = ''
        else:
            rv[tmp_h] += line.strip().replace(' ', '')
    f.close()
    return rv

twinlist class

In [2]:
class twinlist():
    def __init__(self):
        self.type = ""
        self.name = []
        self.value = []
        
    def getdict(self, that, cut = None):
        if isinstance(that,dict) == False:
            raise Exception('invalid getdict')
        self.name = list(that.keys())
        self.value = []
        for i,value in enumerate(self.name):
            self.value.append(that[value])
            if cut != None:
                self.name[i] = value[cut[0]:cut[1]]
        return None
        
    def dictionary(self):
        this = {}
        for i in range (0,len(self.name)):
            key = self.name[i]
            value = self.value[i]
        return this
    
    def __add__(self, another):
        if isinstance(another,twinlist) == False:
            raise Exception('invalid twinlist addition')
        if self.type != another.type:
            raise Exception('types do not match')
        #this is twin twin operation
        this = twinlist()
        this.type = self.type
        this.name += self.name + another.name
        this.value += self.value + another.value
        return this
    
    #simply compare names.the result is elements in one, not in another.
    def __sub__(one, another, notify_missing = 1):
        if isinstance(another,twinlist) == False:
            raise Exception('invalid twinlist substraction')
        if one.type != another.type:
            raise Exception('types do not match')
            
        this = twinlist()
        
        for index, name in enumerate(one.name):
            if name not in another.name:
                this.append_element(name,one.value[index])
        
        if notify_missing == 1:
            for index, name in enumerate(another.name):
                if name not in one.name:
                    print("{}th element {} not in the bigger twinlist".format(index, name))
                
        return this
    
    def __eq__(one, another):
        if one.name != another.name:
            return False
        if one.value != another.value:
            return False
        return True
    
    def __repr__(self):
        return self.__call__()
    
    def __call__(self, short_switch = 0):
        this = ''
        this += 'type : {}'.format(self.type) + '\n'
        
        for i in range (0,len(self.name)):
            if short_switch == 1:
                if len(self.value[i]) > 20:
                    this +=  '{} : {}... (length : {} )\n'.format(self.name[i], self.value[i][:20], len(self.value[i]) )
                    continue
            this += ( '{} : {}'.format(self.name[i], self.value[i]) + '\n')
        return this
    
    #check if there is unknown sequence
    def check(self):
        known = ['A', 'T', 'G', 'C', 'R', 'Y', 'S', 'W', 'K', 'M', 'B', 'V', 'D', 'H', 'N']
        
        for sequence in this.value:
            for i in sequence:
                if i not in known:
                    raise Exception('unknown character: {} in {}'.format(i,sequence))
        print('check successful, nothing to do here')
        return None
    
    #check if there are duplicated values
    def check_duplication(self):
        fine = 0
        match = 0
        for i in range(0,len(self.value)):
            for j in range(0, len(self.value)):
                if self.value[i] == self.value[j]:
                    if i == j:
                        fine = fine + 1
                        continue
                    match = match + 1
            
        print("fine : {}".format(fine))
        print("match : {}".format(match))
    
    #function to trim name
    #more code is needed for trimming only start or only end
    def trim_name(self, start, end):
        for index, name in enumerate(self.name):
            if len(name) < end:
                raise Exception('The 2nd argument, {} is higher than length of {}th name, {}'.format(end, index, name))
            if start < 0:
                raise Exception("invalid 1st argument, {}".format(start))
            
            self.name[index] = name[start:end]
    
    #function to remove gene values' head/tail 
    def trim_sequence(self):
        for index, sequence in enumerate(self.value):
            if sequence.count('-') not in [0,2]:
                raise Exception('strange number of - detected on {}th sequence {}'.format(index,sequence))
            if sequence.count('-') == 0:
                continue
            non1, self.value[index], non2 = sequence.split('-')
        return None
    
    def append_element(self, name, value):
        self.name.append(name)
        self.value.append(value)
    
    def remove_element(self, index):
        self.name = self.name[:index] + self.name[index+1:]
        self.value = self.value[:index] + self.value[index:]
        
    
    def findvalue(self, name):
        for i in range(0,len(self.name)):
            if self.name[i] == name:
                break
        return self.value[i]
    
    def filter_name(self, start, end, keyword):
        this = twinlist()
        this.type = self.type
        for index, name in enumerate(self.name):
            if self.name[index][start:end] == keyword:
                this.append_element(self.name[index], self.value[index])
        #print(this.name[0]) self = this does not work. Is there some data corruption?
        self.name = this.name
        self.value = this.value
    
    #remove elements that are not purely ATGC
    def filter_ATGC(self):
        if self.type != "genome":
            raise Exception("invalid type!")
        this = twinlist()
        for i in range(0, len(self.name)):
            fine = 1
            for j in range(0,len(self.value[i])):
                if self.value[i][j] not in ['A', 'T', 'G', 'C']:
                    fine = 0
                    break
            if fine == 1:
                this.append_element(self.name[i], self.value[i])
        
        return this
        
    
    #returns a twinlist of elements that contain keyword in its value.
    def search_name(self, keyword):
        this = twinlist()
        this.type = self.type
        for index, name in enumerate(self.name):
            if self.name[index].find(keyword) != -1:
                this.append_element(self.name[index], self.value[index])
        return this
    
    #function for finding matching primer
    def perfect_match(self, primer, bound = None):
        if isinstance (primer,twinlist) == False:
            raise Exception('invalid type')
        if (self.type == 'genome' and primer.type == 'primer') == False:
            raise Exception('invalid twinlist.type')
            
        for g_index in range (0,len(self.name)):
            if bound != None:
                if g_index == bound:
                    break
            print('{}. {} \n'.format(g_index+1, self.name[g_index]))
            print('forward:')
            for p_index in range (0,len(primer.name)):
                if self.value[g_index].count(primer.value[p_index]) > 0:
                    print(primer.name[p_index], ' : ', self.value[g_index].count(primer.value[p_index]))
                    
            print('\nbackward:')
            self.value[g_index] = reverse(self.value[g_index])
            for p_index in range (0,len(primer.name)):
                if self.value[g_index].count(primer.value[p_index]) > 0:
                    print(primer.name[p_index], ' : ', self.value[g_index].count(primer.value[p_index]))
            self.value[g_index] = reverse(self.value[g_index])
            print("\n\n")
        return None
        
    def pcr(self, primer_pair, bound = None):
        if isinstance(primer_pair,twinlist) and self.type == 'genome' and primer_pair.type == 'primer pair' == False:
            raise Exception('invalid type')
            
        this = twinlist()
        that = twinlist()
        counter = 0
        mismatch_counter = 0
        for i,name in enumerate(self.name):
            if bound != None:
                if counter == bound:
                    break
            
            genome = self.findvalue(name)
            loc1 = genome.find(primer_pair.value[0])
            genome = genome[loc1:]#cut it
            genome = reverse(genome)#reverse comp it
            loc2 = genome.find(primer_pair.value[1])
            genome = genome[loc2:]#cut it
            genome = reverse(genome)
            
            #pcr is complete
            counter += 1
            print('{}. PCR on {} with f primer: {}, r primer: {}'.format(counter, name, primer_pair.name[0], primer_pair.name[1]))
            #if primer is not found.
            if loc1 == -1 or loc2 == -1:
                mismatch_counter += 1
                print('at least one of the primer does not match the DNA')
                continue
            #if it was successful.
            print('PCR successful at primer position ({}, {}) with amplicons of length {}'.format(loc1,loc2,len(genome)))
            
            this.name.append([name, primer_pair.name[0],primer_pair.name[1]])
            this.value.append(genome)
            
        that.name.append(primer_pair.name)
        that.value.append(1-mismatch_counter/counter)
        
        return this, that
    
    def generous_pcr(self, primer_pair, bound = None):
        if isinstance(primer_pair,twinlist) and self.type == 'genome' and primer_pair.type == 'primer pair' == False:
            raise Exception('invalid type')
            
        this = twinlist()
        that = twinlist()
        counter = 0
        mismatch_counter = 0
        for i,name in enumerate(self.name):
            if bound != None:
                if counter == bound:
                    break
            
            genome = self.findvalue(name)
            loc1 = generous_find(genome, primer_pair.value[0])
            genome = genome[loc1:]#cut it
            genome = reverse(genome)#reverse comp it
            loc2 = generous_find(genome, primer_pair.value[1])
            genome = genome[loc2:]#cut it
            genome = reverse(genome)
            
            #pcr is complete
            counter += 1
            print('{}. PCR on {} with f primer: {}, r primer: {}'.format(counter, name, primer_pair.name[0], primer_pair.name[1]))
            #if primer is not found.
            if loc1 == -1 or loc2 == -1:
                mismatch_counter += 1
                print('at least one of the primer does not match the DNA')
                continue
            #if it was successful.
            print('PCR successful at primer position ({}, {}) with amplicons of length {}'.format(loc1,loc2,len(genome)))
            
            this.name.append([name, primer_pair.name[0],primer_pair.name[1]])
            this.value.append(genome)
            
        that.name.append(primer_pair.name)
        that.value.append(1-mismatch_counter/counter)
        
        return this, that
    
    #convert all element to protein. Change type to protein, return it.(not change itself)
    def genome_to_protein(self):
        if (self.type != "genome"):
            raise Exception('invalid type')
            
        codon = {   'TTT': 'F', 'TTC': 'F', 'TTA': 'L', 'TTG': 'L', 'TCT': 'S',
                    'TCC': 'S', 'TCA': 'S', 'TCG': 'S', 'TAT': 'Y', 'TAC': 'Y',
                    'TGT': 'C', 'TGC': 'C', 'TGG': 'W', 'CTT': 'L', 'CTC': 'L',
                    'CTA': 'L', 'CTG': 'L', 'CCT': 'P', 'CCC': 'P', 'CCA': 'P',
                    'CCG': 'P', 'CAT': 'H', 'CAC': 'H', 'CAA': 'Q', 'CAG': 'Q',
                    'CGT': 'R', 'CGC': 'R', 'CGA': 'R', 'CGG': 'R', 'ATT': 'I',
                    'ATC': 'I', 'ATA': 'I', 'ATG': 'M', 'ACT': 'T', 'ACC': 'T',
                    'ACA': 'T', 'ACG': 'T', 'AAT': 'N', 'AAC': 'N', 'AAA': 'K',
                    'AAG': 'K', 'AGT': 'S', 'AGC': 'S', 'AGA': 'R', 'AGG': 'R',
                    'GTT': 'V', 'GTC': 'V', 'GTA': 'V', 'GTG': 'V', 'GCT': 'A',
                    'GCC': 'A', 'GCA': 'A', 'GCG': 'A', 'GAT': 'D', 'GAC': 'D',
                    'GAA': 'E', 'GAG': 'E', 'GGT': 'G', 'GGC': 'G', 'GGA': 'G',
                    'GGG': 'G', 'TAA': 'stop', 'TAG': 'stop', 'TGA': 'stop'}
        
        that = twinlist()
        for i,name in enumerate(self.name):
            that.value.append("")
            if len(self.value[i]) %3 != 0:
                #raise Exception("sequence not divided by 3 on {}".format(self.name[i]))
                continue
            length = int(len(self.value[i])/3)
            
            if self.value[i][:3] != 'ATG':
                #raise Exception("no start codon on {}".format(self.name[i]))
                continue
            if codon[self.value[i][-3:]] != 'stop':
                #raise Exception("no end codon on {}".format(self.name[i]))
                continue
                
            for j in range(0, length):
                this_codon = codon[self.value[i][3*j : 3*j + 3]]
                
                if this_codon == 'stop':
                    if j+1 == length:
                        continue
                    else:
                        raise Exception("end codon on the middle")
                    
                that.value[i] += this_codon
                
            that.name.append(name)
            
        that.type = "protein"
        return that
    
    def makefasta(self, loc, name = None):
        if self.type not in ['genome', 'protein']:
            raise Exception("unsupported type")
        
        if name == None:
            name = self.type
        name += ".fa"
        name = loc + name
        
        f = open(name, 'w')
        for i, name in enumerate(self.name):
            
            f.write('>' + name + '\n')
            value = self.value[i]
            for j in range(0, int(len(value)/60)):
                f.write(value[j*60:(j+1)*60] + '\n')
            if len(value)%60 != 0:
                f.write(value[(j+1)*60:]+ '\n')
        
        f.close() 

PCR related functions

In [3]:
# The function to get the reverse complementary sequences
rc = {'A': 'T', 'T': 'A', 'G': 'C', 'C': 'G', 'R': 'Y', 'Y': 'R', 'S': 'S', 'W': 'W', 'K': 'M', 'M': 'K', 'B': 'V', 'V': 'B', 'D': 'H', 'H': 'D', 'N': 'N'}
def reverse(tmp_seq):
    # from the end of input sequence to the beginning, 
    # replace each nucleotide to its complementary one.
    #
    # The single line code at the end of this function is equivalent to the following 4 lines.
    # rv = []
    # for tmp_n in tmp_seq[::-1]:
    #     rv.append(rc[tmp_n])
    # return ''.join(rv)
    
    return ''.join([rc[x] for x in tmp_seq[::-1]])

#print(reverse('AATTGGCC'))
#print(reverse('...'))

In [4]:
#This is more generous than string find method. It'll allow up to 1 mismatch of primer to sequence.
def generous_find(seq, pr):
    for i in range(0, len(seq) - len(pr)):
        focus = seq[i:i+len(pr)]
        mismatch = 0
        for j in range(0,len(pr)):
            if mismatch > 1:
                break
            if focus[j] != pr[j]:
                mismatch += 1
        if mismatch < 2:
            return i
    return -1

#generous_find('aaaaaaaaaababaaaaaaaaaa', 'bbb')

# The Work

# data filtering

read fasta

In [5]:
loc1 = "data/2019nCoV_genomes.2020_04_14.fa"
data = read(loc1)

In [6]:
sample_genomes = twinlist()
sample_genomes.getdict(data)
sample_genomes = sample_genomes.search_name("USA")
#sample_genomes.trim_name(0,8)
len(sample_genomes.name)

667

In [7]:
print(sample_genomes(1))

type : 
MT325590 |Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/NC_0031/2020| complete genome : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325640 |Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/MA_3653/2020| complete genome : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325606 |Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/SC_3569/2020| complete genome : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325607 |Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/SC_3575/2020| complete genome : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325608 |Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/SC_6370/2020| complete genome : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325609 |Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/UT_8199/2020| complete genome : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325610 |Severe acute res

I was able to check that the DNA samples had 3 different categories, which are complete genome, complete cds, and partial genomes.

So I created a function that can separate elements with names with searched keyword, and implemented subtraction between twinlists, which will return elements that are only included in the first operand.

In [8]:
genomes = sample_genomes.search_name("complete genome")
cds = sample_genomes.search_name("complete cds")
len(genomes.name) + len(cds.name)

639

In [9]:
partial_genomes = sample_genomes - genomes
partial_genomes = partial_genomes - cds

In [10]:
print(genomes(1))

type : 
MT325590 |Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/NC_0031/2020| complete genome : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325640 |Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/MA_3653/2020| complete genome : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325606 |Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/SC_3569/2020| complete genome : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325607 |Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/SC_3575/2020| complete genome : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325608 |Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/SC_6370/2020| complete genome : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325609 |Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/UT_8199/2020| complete genome : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325610 |Severe acute res

In [11]:
print(cds(1))

type : 
MT326031 |Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/WA-UW-2008/2020 ORF1ab polyprotein (ORF1ab)| ORF1a polyprotein (ORF1ab)| surface glycoprotein (S)| ORF3a protein (ORF3a)| envelope protein (E)| membrane glycoprotein (M)| ORF6 protein (ORF6)| ORF7a protein (ORF7a)| ORF7b (ORF7b)| ORF8 protein (ORF8)| nucleocapsid phosphoprotein (N)| and ORF10 protein (ORF10) genes| complete cds : TAATTAATAACTAATTACTG... (length : 29775 )
MT326080 |Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/WA-UW-1963/2020 ORF1ab polyprotein (ORF1ab) and ORF1a polyprotein (ORF1ab) genes| partial cds; surface glycoprotein (S)| ORF3a protein (ORF3a)| and envelope protein (E) genes| complete cds; M gene| partial sequence; ORF6 gene| complete sequence; and ORF7a protein (ORF7a)| ORF7b (ORF7b)| ORF8 protein (ORF8)| nucleocapsid phosphoprotein (N)| and ORF10 protein (ORF10) genes| complete cds : GCTTAGTAGAAGTTGAAAAA... (length : 29379 )
MT326077 |Se

In [12]:
print(partial_genomes(1))

type : 
MT263385 |Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/WA-UW302/2020| partial genome : AGAAAACACACGTCCAACTC... (length : 29610 )
MT263407 |Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/WA-UW327/2020| partial genome : TGTCCCTGGTTTCAACGAGA... (length : 29575 )
MT263401 |Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/WA-UW321/2020| partial genome : TATCAGAGGCACGTCAACAT... (length : 29473 )
MT263461 |Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/WA-UW382/2020| partial genome : CTTTAAAATCTGTGTGGCTG... (length : 29828 )
MT263427 |Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/WA-UW347/2020| partial genome : GTCACTCGGCTGCATGCTTA... (length : 29818 )
MT263409 |Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/WA-UW329/2020| partial genome : CCGTGGAGGAGGTCTTATCA... (length : 29443 )
MT263389 |Severe acute res

I will only use complete genomes, as I'm worried if the difference between cds and partial genomes, irrelevant to mutation of genome, can affect the result.

In [13]:
len(genomes.name)

560

I have 560 complete genomes from USA

Next, I don't want sequences that contain any base other than A,T,G,C.

In [15]:
genomes.type = "genome"
genomes = genomes.filter_ATGC()

In [16]:
len(genomes.name)

278

This leaves only 278 genes

In [17]:
genomes.type = 'genome'
genomes.makefasta("./", "USA_complete_genomes")

In [18]:
genomes2 = genomes

# data processing

In [19]:
loc2 = "./USA_complete_genomes.fa"
data = read(loc2)
genomes = twinlist()
genomes.getdict(data)

In [20]:
#Check whether this shortcut works.
#genomes == genomes2

True

In [21]:
print(genomes(1))

type : 
MT325590 |Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/NC_0031/2020| complete genome : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325640 |Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/MA_3653/2020| complete genome : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325606 |Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/SC_3569/2020| complete genome : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325607 |Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/SC_3575/2020| complete genome : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325609 |Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/UT_8199/2020| complete genome : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325611 |Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/VA_6352/2020| complete genome : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325616 |Severe acute res

I remove all the information after the name, as I do not need them.

The csv holds some data that can be used, which is better organized. I think I'm fine with that.

In [22]:
genomes.trim_name(0,8)
print(genomes(1))

type : 
MT325590 : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325640 : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325606 : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325607 : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325609 : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325611 : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325616 : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325618 : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325619 : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325620 : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325622 : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325623 : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325624 : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325599 : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325600 : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325601 : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325615 : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325617 : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325625 : ATTAAAGGTTTATACCTTCC... (le

I check whether there are same sequences among the genomes.

In [23]:
genomes.check_duplication()

fine : 278
match : 522


So, there are some sequences that are identical to each other.
For now, I don't need identical sequences, So I will make a twinlist with unique sequences.

In [24]:
unique_genomes = twinlist()
for i, sequence in enumerate(genomes.value):
    if sequence in unique_genomes.value:
        continue
    else:
        unique_genomes.append_element(genomes.name[i], sequence)

In [25]:
print(unique_genomes(1))

type : 
MT325590 : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325640 : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325606 : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325607 : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325609 : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325611 : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325618 : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325619 : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325622 : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325599 : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325615 : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325625 : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT324684 : ATTAAAGGTTTATACCTTCC... (length : 29889 )
MT325577 : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325586 : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325592 : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325594 : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325598 : ATTAAAGGTTTATACCTTCC... (length : 29882 )
MT325626 : ATTAAAGGTTTATACCTTCC... (le

testing unique_genomes

In [26]:
unique_genomes.check_duplication()
test = twinlist()
test = unique_genomes.__sub__(genomes, 0)
print(test)

fine : 215
match : 0
type : 



Only 215 genes are left, which I think I can work with.

Now, I'll draw phylogenetic tree with unique_genomes, to get candidates of original genomes.

In [27]:
unique_genomes.type = "genome"
unique_genomes.makefasta("./", "unique_genomes")

# Drawing tree

What should I do now?

In [1]:
print("""
I need a data structure, that can hold not only name and sequence, but also some additional informations of the genome.
For example, I need time and location of each sample.

Actually, I don't think I can make a structure that can freely add new rows and columns of data.
I don't need to go the hard way for this simple case, I can do it simple.
I can make new lists that can hold the information that is needed.
For example, I can do "self.time = [], self.location = []"
For now, I can make a new method to generate these instance variables, fill them with None, and change their value with given csv file.
After that, I can filter the elements that does not have either time or location info, or elements that have wrong format. That will leave me legit data, that can be used throughout the projec.
""")


I need a data structure, that can hold not only name and sequence, but also some additional informations of the genome.
For example, I need time and location of each sample.

Actually, I don't think I can make a structure that can freely add new rows and columns of data.
I don't need to go the hard way for this simple case, I can do it simple.
I can make new lists that can hold the information that is needed.
For example, I can do "self.time = [], self.location = []"
For now, I can make a new method to generate these instance variables, fill them with None, and change their value with given csv file.
After that, I can filter the elements that does not have either time or location info, or elements that have wrong format. That will leave me legit data, that can be used throughout the projec.



I can search csv to find locations of these.
Now, I should get csv data in form of list or something,
compare each element of my twinlist to this list, to check if there is legit location.
For assignment 1, I need sequences with exact location.
After I filter only sequences with legit location info, I can then further file up same genes at same location.
And then, I can make pyrogenetic tree, analyze it, which ends question 1

How do I add time axis to pyrogenetic tree? I'm not sure...
I think I can also use time info in the list to make a more accurate pyrogenetic tree...
But how?

Then, for question 2, I need to filter this 'genomes' to get genomes that have legit time info, from csv file.
Then, with this time info, and by finding number of mutations, I can calculate the mutation rate.
But what is original one? From what do I compare?
Can I find it from question 1?

I can do the same on each protein coding sequence, by finding sequences following steps of former assignments.
Then, for each list of sequences that code each protein, I can calculate mutation rate, compare it between proteins.
Or, just follow what lecture says.

# New Idea

In [5]:
print("""
question 1 is about the path of infection. The tree first needs infection path. But after the path is given, by path I mean the order and time of samples, there is nothing more needed, such as name of sequence. All that is needed as leaf value, is location.
So after getting the tree, all I need is to change the name to location, and visualize that. with colored different sized circles.

question 2 and 3 is about mutation rate. Only difference is whether to use the whole genome or cut it into protein coding parts. The steps to find specific locations are time-consuming, but I can do that. Possibly easily.

given codes, how do I find mutation rates?
There are multipe candidates of origin of mutation. Corona started from many different points of USA.
But, it should be at the start of largest branch, the start of each large branch.
There must be multiple 'different' points. How do I choose the original one?
I should use the earliest ones, but not only them. They may have all been, or mostly, mutated. I should consider first 2, 3 earliest groups of samples at same time, give them weight, such as 4,2,1, add up the score for each sequence, and find the origins.

Theere are multiple origins.
From there, mutations can be found by comparing the sequenes to the assigned origin.
Now, I should find rigorus definition about mutation rate, which must be function of time.
I can find average mutation rate, out of the graph.
But if there are more patients, it is natural that there is higher mutation.
How is mutation rate defined?
""")


question 1 is about the path of infection. The tree first needs infection path. But after the path is given, by path I mean the order and time of samples, there is nothing more needed, such as name of sequence. All that is needed as leaf value, is location.
So after getting the tree, all I need is to change the name to location, and visualize that. with colored different sized circles.

question 2 and 3 is about mutation rate. Only difference is whether to use the whole genome or cut it into protein coding parts. The steps to find specific locations are time-consuming, but I can do that. Possibly easily.

given codes, how do I find mutation rates?
There are multipe candidates of origin of mutation. Corona started from many different points of USA.
But, it should be at the start of largest branch, the start of each large branch.
There must be multiple 'different' points. How do I choose the original one?
I should use the earliest ones, but not only them. They may have all been, or most

In [3]:
print("""
I should first draw phyrogenetic tree. That's the first step.
Then, using the tree and sample dates, I should build a new phylogenetic tree.
However, how do I generate a new tree? Before that, what tree should I make?
One sequence can be ancestor of another. The older patient may have been found later than the new patient, the ancester's found date may be later than newer patients'.


If I assume the release data = mutation date,

Um, let me state my problem here.
There are patients A,B,C.
A has mutation a.
B has mutation a and b,
C has mutation a and c on the virus.
Then, is A common ancestor of B and C?

Before that, should be there a common ancestor between B and C?
What is mutation rate? Number of total mutations in the whole sample?
""")


I should first draw phyrogenetic tree. That's the first step.
Then, using the tree and sample dates, I should build a new phylogenetic tree.
However, how do I generate a new tree? Before that, what tree should I make?
One sequence can be ancestor of another. The older patient may have been found later than the new patient, the ancester's found date may be later than newer patients'.


If I assume the release data = mutation date,

Um, let me state my problem here.
There are patients A,B,C.
A has mutation a.
B has mutation a and b,
C has mutation a and c on the virus.
Then, is A common ancestor of B and C?

Before that, should be there a common ancestor between B and C?
What is mutation rate? Number of total mutations in the whole sample?



In [4]:
print("""
Nothing is sure whether each sample can be a common ancestor or not.
There only is possibility that each leaf can be the node.
""")


Nothing is sure whether each sample can be a common ancestor or not.
There only is possibility that each leaf can be the node.



In [2]:
print("""
I can make a phyrogenetic tree with time scale on each leafs.

The tree is incomplete, actually the tree has no branch values. The branches are void.

Is every branch of every phyrogenetic tree void?

How do I choose the time of node? Well it is safe to make the node as late as possible. That way, if there is a common ancestor, the ancestor equals the leaf right next to the node, at the same time. This way, every leaf, has a chance to become a node, the common ancestor, but also may not be.

I have a tree with all leafs and nodes(branches) given a specific time value.
So I should only use dna samples with intact release date.
Maybe for visualization, I should move the leaf half step forward from the node at the same moment.
""")


I can make a phyrogenetic tree with time scale on each leafs.

The tree is incomplete, actually the tree has no branch values. The branches are void.

Is every branch of every phyrogenetic tree void?

How do I choose the time of node? Well it is safe to make the node as late as possible. That way, if there is a common ancestor, the ancestor equals the leaf right next to the node, at the same time. This way, every leaf, has a chance to become a node, the common ancestor, but also may not be.

I have a tree with all leafs and nodes(branches) given a specific time value.
So I should only use dna samples with intact release date.
Maybe for visualization, I should move the leaf half step forward from the node at the same moment.

